# T-Mobile Alarm Analytics v 3.0

## version: 2020-10-05

## Loads Clusters identified during prior processing

### Does not load individual alarms

In [ ]:
import csv
import random
import json
from time import process_time 
import http.client, urllib.parse


from random import randint
from py2neo import Graph
 
communitiesFileName = '/Users/markquinsland/Documents/tmobile/alarm_analytics_v3/data/alarm_clusters_8-13.csv'

## sample rows of input file

``` text
cluster,count,"exec_time",objects
7,166,"2020-04-22 00:06:18","- SNMP SC SOAP Prov. Latency by AM Card::Mavenir_CDB"
7,166,"2020-04-22 00:06:18","/customer-loan/v1/loan-bundle-quotes: Health::Device-Finance"


```

In [ ]:
# connect to neo

  
try:
    graph = Graph("bolt://localhost", auth=("neo4j", "nimda"))
    print("Nodes: ", len(graph.nodes))
    print("Relationships: ", len(graph.relationships))
except Exception as e:
    print(type(e))
    print(e)


In [ ]:


# indexes are in first map, constraints are in second map, TRUE will drop all existing constraints and indices
# this API call does not currently support creating compound indexes such as Node Keys.  They must be created separately.

schemaIndexCypherStmt = ''' 

CALL apoc.schema.assert(
    {ManagerClass:['type','style'] }, {
      Cluster:['id'],
      TimeBucket:['time'],
      Event:['id'],
      AlarmClass:['id'],
      ManagerClass:['id']
      }, TRUE)
'''
graph.run(schemaIndexCypherStmt).to_table()



In [ ]:
## generate node key constraints

cypherStmt = ''' 

CREATE CONSTRAINT alarm_key
ON (al:Alarm) ASSERT (al.id,al.time) IS NODE KEY;

     
'''
graph.run(cypherStmt).data()

In [ ]:
cypherStmt =  '''
USING PERIODIC COMMIT 10000
LOAD CSV WITH HEADERS FROM "file:///''' + communitiesFileName + '''" AS row
// with row limit 100
with row,split(row['objects'],"::") as parts, left (row.exec_time,10) as execDate

// return parts [0],execDate, execDate + "_" +row['cluster']  as id
MERGE (cl:Cluster {id:execDate + "_" +row['cluster'] })
  set cl.idCount = toInteger(row.count)
 MERGE (ac:AlarmClass {id:parts[0]})
 MERGE (mc:ManagerClass {id:parts[1]})
 MERGE (ev:Event {id:row['objects']})
 MERGE (ev)<-[:HAS_EVENT]-(cl)
 MERGE (ev)-[:OF_CLASS]->(ac)
 MERGE (ac)-[:IS]->(mc)
'''

graph.run(cypherStmt).to_table()

## Loading individual alarms

In [ ]:
def loadAlarms (directoryName, fileName):
    
    
    cypherStmt =  '''
    // USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM "file:///''' + directoryName + fileName + '''" AS row
    //with row limit 100
    with row,CASE WHEN toInteger(row.created_date) > 15910100000 THEN toInteger(row.created_date) ELSE toInteger(row.created_date) * 1000 END  as ts
    MERGE (al:Alarm {id: row['alert_name'] + "::" + row['manager_class'],time:datetime({ epochMillis: ts }) })   
    set al.fileName = "''' + fileName +      '''", 
    al.timestamp = ts
   
   with al,row
    
    Match (ev:Event {id: row['alert_name'] + "::" + row['manager_class']})
    MERGE (ev)<-[:IS]-(al)
    return count(*) as count
    '''

    #print (cypherStmt)
    count =  graph.run(cypherStmt).evaluate() or 0
    print (fileName, count)
    return count

In [ ]:
def loadAlarms_old (fileName):
    cypherStmt =  '''
    USING PERIODIC COMMIT 10000
    LOAD CSV WITH HEADERS FROM "file:///''' + alarmsDirectoryName + fileName + '''" AS row
    with row, toInteger(round(toInteger(row.created_date)/300000*300000  )) as timeBucketMillis

    MERGE (tb:TimeBucket {time:datetime({ epochMillis: timeBucketMillis }) })   
    with tb,row
    Match (ev:Event {id: row['alert_name'] + "::" + row['manager_class']})
    MERGE (ev)-[:IN]->(tb)
    return count(*)
    '''

    graph.run(cypherStmt).to_table()

# Load Alarm files from Directory

In [ ]:
import os

def files(path):
    for file in os.listdir(path):
        if os.path.isfile(os.path.join(path, file)):
            yield file
fileCount = 0
rowCount = 0           
for file in files(alarmsDirectoryName):
    if file == '.DS_Store':
        print ("skip", file)
        continue
    fileCount += 1
    print ("loading", file)
    count = loadAlarms(alarmsDirectoryName, file)
    print ("rows=", count)
    rowCount += count
    
print ("total files", fileCount)    
print ("total rows", rowCount)

# create time buckets


## delete existing timestamps  - optional


In [ ]:

cypherStmt =  ''' 

MATCH  (tb:TimeBucket )
DETACH DELETE tb
return   count(*) as bucketCount  
'''


print (graph.run(cypherStmt).to_table())




# Preview Bucket Size 
### python dataframe example

In [ ]:
bucketSizeMinutes = 5


cypherStmt =  ''' 


MATCH (ev:Event)<-[:IS]-(al:Alarm) with ev, datetime({ epochMillis: al.time.epochMillis / $bucketMillis * $bucketMillis }) as bucketTime
// MERGE (tb:TimeBucket {time:bucketTime})
//MERGE (ev)-[:OCCURRED]->(tb)

return bucketTime, count(*) as bucketSize order by bucketSize desc
'''

bucketMillis = bucketSizeMinutes * 60 * 1000
eventStats = graph.run(cypherStmt,{'bucketMillis':bucketMillis}).to_data_frame()
eventStats[["bucketSize"]].describe()




In [ ]:
## Update the database with ts

In [ ]:
bucketSizeMinutes = 10


cypherStmt =  ''' 

MATCH (ev:Event)<-[:IS]-(al:Alarm) with ev,al.time as alarmTime, datetime({ epochMillis: al.time.epochMillis / $bucketMillis * $bucketMillis }) as bucketTime
MERGE (tb:TimeBucket {time:bucketTime})
MERGE (ev)-[oc:OCCURRED]->(tb)
    set oc.time = alarmTime

return bucketTime, count(*) as bucketSize order by bucketSize desc
'''

bucketMillis = bucketSizeMinutes * 60 * 1000
eventStats = graph.run(cypherStmt,{'bucketMillis':bucketMillis}).to_data_frame()
eventStats[["bucketSize"]].describe()



In [ ]:
cypherStmt = ''' 
    MATCH (e1:Event)-[:OCCURRED]->(tb:TimeBucket)<-[:OCCURRED]-(e2:Event) 
    WHERE exists ((e1)-[:CO_OCCURS]-(e2))
    with  id(e1) as e1Id, id(e2) as e2Id, count(id(tb)) as sharedTimeBuckets
    limit $limit
    return *
'''
 
eventStats = graph.run(cypherStmt,{'limit':500}).to_data_frame()
eventStats[["sharedTimeBuckets"]].describe()


In [ ]:
cypherStmt = ''' 

MATCH (cl:Cluster)-[:HAS_EVENT]->(ev:Event)
return id(cl) as clusterId, count(distinct id(ev)) as events order by clusterId
 
 

'''
graph.run(cypherStmt ).to_table()


# Get Event Sequence Info
Using the timebuckets for any given time range, determine which events precede other event within each time buckets,
then aggregate the counts.  

Optionally use a minimum percentage in order to include a value. Set it to 0 to include all records.

The first query obtains all events tied to a cluster and sorts them in chronological order - for each timebucket. This is important because it limits the number of potential events to just those that are in the cluster.   

To process the results, the clusters are no longer important.  We're looking to see how often events preceed one another and the aggregation is at the event level, not the cluster level.   




In [ ]:

startDate = '2020-05-01'
endDate = '2020-05-31'
evPairs = {}
evCounts = {}
batch = []
minPct = 20

cypherStmt = ''' 


MATCH (cl:Cluster)-[:HAS_EVENT]->(ev:Event)-[r:OCCURRED]->(tb:TimeBucket)
where datetime($startDate) <=tb.time <= datetime( $endDate)

with * 
limit $limit
 with tb.time as tbTime, id(cl) as clusterId, id(ev) as eventId, r.time as evTime 
with * order by tbTime, clusterId, evTime  
with tbTime, clusterId, collect(eventId) as events
return  clusterId,  collect (events) as tbEvents

'''
results = graph.run(cypherStmt,{'startDate':startDate,'endDate':endDate,'limit':5000000})

x = 0
for record in results:
    #if x>50:
    #   break
    x+=1
    clusterId = record['clusterId']
    tbEvents = record['tbEvents']
    #print (x,'clustId',clusterId, 'tbEventsCount',len(tbEvents), tbEvents)
    print (x,'clustId',clusterId, 'tbEventsCount',len(tbEvents) )
    loadClusterSequences (clusterId, tbEvents, minimumPct)

    
    
#print ('*********','evCounts', evCounts)
for key in evPairs:
    parts = key.split('_')
    count = evPairs[key]
    evCount =  evCounts[int(parts[0])]
    # print ('*********','evCounts', evCounts)
    matchPct = (evPairs[key] / evCounts[int(parts[0])]) * 100
                                        
    print ('key', key,'count',count, 'pct', matchPct)
    if matchPct < minPct:
        continue
    
    batch.append(
        {'fromId':int(parts[0]),
         'toId':int(parts[1]),
         'count':evPairs[key], 
         'pct':matchPct, 
         'eventCount':evCounts[int(parts[0]) ]}
         )

print ('**** batch ',batch)

createRelationshipsCypherStmt = '''  

 UNWIND $relationships as row
    MATCH (fr:Event  ) where id(fr) = row['fromId']
    MATCH (to:Event) where id(to) = row['toId']
    MERGE (fr)-[rel:PRECEDES]->(to)
        SET rel.precedesCount = row['count'],
            rel.eventCount = row['eventCount'],
            rel.pct =  toInteger() (row.precedesCount * 10000 / row.eventCount) /  100 )
    return count(*) 
'''

graph.run(createRelationshipsCypherStmt,{'relationships':batch}).to_table()


# determine cluster sequences for each timebucket

In [ ]:
def loadClusterSequences (clusterId, tbEvents, minPct):
    
   
    
    tbCount = len(tbEvents)
    
    global evPairs
    global evCounts
    global batch 
    
    ''' 
    loop through all of the events in the timebucket.  they are sorted, oldest first.  for each event,
    create a sequence pair for each of the events following it, not just the ones immediately following it.
    This will help address problems like A,B,C then A,C,B - we want to know that both C and B follow A twice.
    
    '''
    for tb in tbEvents:
        tbSize = len(tb)
        #print (tbCount, len(tb))
        for x in range (0,tbSize):
            #print (tbCount,tb[x], tbSize)
            if tb[x] in evCounts:
                evCounts [tb[x]] +=1
            else:
                evCounts [tb[x]]=1

            if x+1 >= tbSize:
                continue
            key = str(tb[x])+ '_' + str(tb[x+1])
            #print ("** ",x, key)
            if key in evPairs:
                evPairs [key] +=1
            else:
                evPairs [key]=1


   

In [ ]:
cypherStmt = ''' 
    MATCH (e1:Event)-[:OCCURRED]->(tb:TimeBucket)<-[:OCCURRED]-(e2:Event) 
  
    WHERE id(e1) > id(e2)
      and exists ((e1)-[:CO_OCCURS]-(e2))
    with  e1,e2, count(tb) as sharedTimeBuckets
    //limit $limit
   // MERGE (e1)-[rel:CO_OCCURS]-(e2)
    // set rel.tb_count = sharedTimeBuckets
    return * limit 100
'''
 
print(graph.run(cypherStmt).to_table())



# Older queries kept but not verified as still current

## How many events are in each cluster?

In [ ]:
cypherStmt = ''' 
    MATCH (cl:Cluster)
    return cl.idCount as eventCount 
'''
clusterStats = graph.run(cypherStmt).to_data_frame()
clusterStats[["eventCount"]].describe()


## Calculate the number of clusters each event belongs to

In [ ]:
cypherStmt = ''' 
    match (e:Event)<-[:HAS_EVENT]-(:Cluster)
    with e,count(*) as clusterCount 
    set e.clusterCount = clusterCount
    return e.id as eventid, clusterCount
'''
eventStats = graph.run(cypherStmt).to_data_frame()
eventStats[["clusterCount"]].describe()


## Calculate the total number of siblings each event has - across all clusters

In [ ]:
# get the total number of distinct events that co-occur (in any related cluster) with each event

cypherStmt = '''  
    MATCH (e1:Event)<-[:HAS_EVENT]-(c)-[:HAS_EVENT]->(e2)
    with e1.id as eventId,  count(distinct e2.id) as siblingCount
    return *
'''

siblingDF = graph.run(cypherStmt).to_data_frame()
siblingDF[["siblingCount"]].describe()


In [ ]:
cypherStmt = ''' 
MATCH (e1:Event) <-[:HAS_EVENT]-(c)-[:HAS_EVENT]->(e2)
    with e1, e1.clusterCount as clusterCount, e2, e2.clusterCount as e2ClusterCount, count(id(c)) as coCount
   // with e1, clusterCount, e2, e2ClusterCount,count(cId) as coCount
    where coCount > $minCoCount
    with *, toInteger((toFloat(coCount)/clusterCount)*100) AS e1Pct, toInteger((toFloat(coCount)/e2ClusterCount)*100) AS e2Pct
    where e1Pct >= $minPct or e2Pct >= $minPct
    and id(e1) < id(e2)
    return e1.clusterCount as e1Count, e1Pct, e2.clusterCount as e2Count, e2Pct, coCount, left(e1.id,50) as event1, left(e2.id,50) as event2   order by  coCount desc
    limit $limit
'''
graph.run(cypherStmt,{'limit': 300, 'minPct':75, 'minCoCount':5}).to_table()

# Create CO_OCCURS Relationships
These indicate that 2 event nodes are both in at least 1 cluster.   The number of co-occurrences will be stored as a property value called 'count'.   This property value can then be used to determine the co-occurrence percentage for each of the 2 nodes.  The higher the ratio of co-occurrences to the number of clusters (likely different for each of the 2 nodes), the higher the correlation is between the 2 events.

In [ ]:
cypherStmt = ''' 
    match(:Event)-[rel:CO_OCCURS]-()
    delete rel
    return count (rel ) as deleted
'''
graph.run(cypherStmt).to_table()

In [ ]:
# calculate and store the correlation percentages of the relationships between events

cypherStmt =  ''' 
MATCH (e1:Event) <-[:HAS_EVENT]-(c)-[:HAS_EVENT]->(e2)
   where id(e1) < id(e2)
    with e1, e2,count(c) as coCount
    with *, (toFloat(coCount)/e1.clusterCount)*100 AS e1Pct, (toFloat(coCount)/e2.clusterCount)*100 AS e2Pct
    where e1Pct >= $minPct and e2Pct >= $minPct 
 merge (e1)-[rel:CO_OCCURS]->(e2)
    set rel.count = coCount, rel.e1Pct = toInteger(e1Pct), rel.e2Pct = toInteger(e2Pct)
RETURN count(*) as created
'''
graph.run(cypherStmt,{ 'minPct':60}).to_table()

# TimeBucket Queries

## Description:  
The goal of these queries is to determine which :Event nodes frequently occur.  

Log files from approximately 30 days of activity have been provided for testing.  Each alarm record
is linked to a :TimeBucket node after rounding the :Alarm time to the nearest 5 minutes.

After loading the 

### Relationships Beetween Clusters, Events, and Alarms

<img src="resources/time_bucket_overview.png" width="800">

In [ ]:
# get the total number of distinct time buckets for each event

cypherStmt = '''  
    MATCH (e1:Event)-[:IN]-(tb) 
    with e1.id as eventId,  count(distinct id(tb)) as bucketCount
    return *
'''

siblingDF = graph.run(cypherStmt).to_data_frame()
siblingDF[["bucketCount"]].describe()

In [ ]:
# get the number of events linked to each timebucket

cypherStmt = '''  
    MATCH (e1:Event)-[:IN]->(tb) 
    with id(tb) as bucketId,  count(distinct id(e1)) as eventCount
    return bucketId, eventCount
'''

siblingDF = graph.run(cypherStmt).to_data_frame()
siblingDF[["eventCount"]].describe()

In [ ]:
## get the events that are in the same cluster and in the same time buckets

cypherStmt = '''  
    MATCH (e1:Event)-[:IN]->(tb)<-[:IN]-(e2:Event)
    where exists( (e1) <-[:HAS_EVENT]-(:Cluster)-[:HAS_EVENT]->(e2))
    return e1.id as ev1, e2.id as ev2, count(*) as count
'''

siblingDF = graph.run(cypherStmt).to_data_frame()


In [ ]:
siblingDF[["count"]].describe()

In [ ]:
#sorting data frame by name 
siblingDF.sort_values("count", axis = 0, ascending = False, 
                 inplace = True)
siblingDF[1:30]